In [1]:
import sys
import os
from typing import Optional
from tqdm import tqdm
sys.path.append(os.path.abspath('../'))

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import ttest_ind
import seaborn as sns
import os
from IPython.display import display, HTML
import torch

from utils.plot_utils import *
from utils.utils import *

pd.set_option('display.max_rows', 500)

In [2]:
count_inf = -10
experiment_dir = "../results/JailbreakLikelihoodRatio/JailbreakLikelihoodRatio_aligned=meta-llama-Llama-3.1-8B-Instruct_unaligned=grimjim-Llama-3.1-8B-Instruct-abliterated_via_adapter_dataset=Mechanistic-Anomaly-Detection-llama3-jailbreaks_num_samples=None"

In [3]:
aligned_jailbreak_logprobs = torch.load(f"{experiment_dir}/aligned_jailbreak_logprobs.pt",weights_only=True)
aligned_benign_logprobs = torch.load(f"{experiment_dir}/aligned_benign_logprobs.pt",weights_only=True)

unaligned_jailbreak_logprobs = torch.load(f"{experiment_dir}/unaligned_jailbreak_logprobs.pt",weights_only=True)
unaligned_benign_logprobs = torch.load(f"{experiment_dir}/unaligned_benign_logprobs.pt",weights_only=True)

In [5]:
aligned_jailbreak_logprobs[0].keys()

dict_keys(['prompt', 'completion', 'original_split', 'context', 'input_tokens', 'response_tokens', 'response_string', 'no', 'user_logprobs', 'asst_logprobs', 'gen_logprobs'])

In [6]:
aligned_jb_logprobs_df = pt_to_df(None, aligned_jailbreak_logprobs, count_inf)
unaligned_jb_logprobs_df = pt_to_df(None, unaligned_jailbreak_logprobs, count_inf) 

aligned_benign_logprobs_df = pt_to_df(None, aligned_benign_logprobs, count_inf)
unaligned_benign_logprobs_df = pt_to_df(None, unaligned_benign_logprobs, count_inf) 

## User and asst logprobs

In [63]:
user_jb, user_benign = return_diffs(unaligned_jb_logprobs_df, aligned_jb_logprobs_df,
                          unaligned_benign_logprobs_df, aligned_benign_logprobs_df,prefix="user")
asst_jb, asst_benign = return_diffs(unaligned_jb_logprobs_df, aligned_jb_logprobs_df,
                          unaligned_benign_logprobs_df, aligned_benign_logprobs_df,prefix="asst")

In [65]:
lst_of_columns = []
for column in ["diff","aligned_logprobs","unaligned_logprobs"]:
    lst_of_columns.extend(
        [
            (user_jb
             .groupby("original_split")[column]
             .apply(lambda x:compute_tprs(np.array(x), user_jb[column])
                    .set_index("FPRs").loc[0.01]
                    )
             .rename(f"user_{column}")
            ),
            (asst_jb
             .groupby("original_split")[column]
             .apply(lambda x:compute_tprs(np.array(x), asst_benign[column])
                    .set_index("FPRs").loc[0.01]
                    )
             .rename(f"asst_{column}")
            )
        ]
    )

In [72]:
df_total = pd.concat(lst_of_columns, axis=1).T
df_total

original_split,harmful_autodan,harmful_best_of_n,harmful_gcg,harmful_human_mt,harmful_misc,harmful_msj,harmful_pair,harmful_prefill
,TPRs,TPRs,TPRs,TPRs,TPRs,TPRs,TPRs,TPRs
user_diff,0.000000,0.015707,0.01,0.000000,0.0375,0.00000,0.011364,0.020305
asst_diff,0.082051,0.047120,0.03,0.000000,0.1000,0.00000,0.060606,0.050761
user_aligned_logprobs,0.025641,0.005236,0.03,0.000000,0.0000,0.00000,0.000000,0.020305
asst_aligned_logprobs,0.205128,0.136126,0.15,0.570447,0.2000,0.00625,0.015152,0.416244
user_unaligned_logprobs,0.020513,0.005236,0.04,0.000000,0.0000,0.00000,0.000000,0.025381
asst_unaligned_logprobs,0.210256,0.188482,0.12,0.560137,0.2000,0.00000,0.030303,0.360406
